In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, pairs_from_covisibility, pairs_from_retrieval
from hloc import colmap_from_nvm, triangulation, localize_sfm, visualization

In [5]:
dataset = Path('datasets/Droid_SLAM/')  # change this if your dataset is somewhere else
images = dataset / 'left_images/'

outputs = Path('outputs/Droid_SLAM/')  # where everything will be saved
sfm_pairs = outputs / 'pairs-db-covis20.txt'  # top 20 most covisible in SIFT model
loc_pairs = outputs / 'pairs-query-netvlad20.txt'  # top 20 retrieved by NetVLAD
reference_sfm = outputs / 'sfm_superpoint+superglue'  # the SfM model we will build
results = outputs / 'uHuman_hloc_superpoint+superglue_netvlad20.txt'  # the result file

# list the standard configurations available
print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')

Configs for feature extractors:
{'cosplace': {'model': {'name': 'cosplace'},
              'output': 'global-feats-cosplace',
              'preprocessing': {'resize_max': 1024}},
 'd2net-ss': {'model': {'multiscale': False, 'name': 'd2net'},
              'output': 'feats-d2net-ss',
              'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'dir': {'model': {'name': 'dir'},
         'output': 'global-feats-dir',
         'preprocessing': {'resize_max': 1024}},
 'disk': {'model': {'max_keypoints': 5000, 'name': 'disk'},
          'output': 'feats-disk',
          'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'netvlad': {'model': {'name': 'netvlad'},
             'output': 'global-feats-netvlad',
             'preprocessing': {'resize_max': 1024}},
 'openibl': {'model': {'name': 'openibl'},
             'output': 'global-feats-openibl',
             'preprocessing': {'resize_max': 1024}},
 'r2d2': {'model': {'max_keypoints': 5000, 'name': 'r2d2'},
        

In [3]:
# pick one of the configurations for image retrieval, local feature extraction, and matching
# you can also simply write your own here!
retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Extract local features for database and query images

In [4]:
features = extract_features.main(feature_conf, images, outputs)

[2023/04/16 15:14:28 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2023/04/16 15:14:28 hloc INFO] Found 328 images in root datasets/Droid_SLAM/left_images.


Loaded SuperPoint model


100%|██████████| 328/328 [04:19<00:00,  1.26it/s]
[2023/04/16 15:18:48 hloc INFO] Finished exporting features.


## Find image pairs via image retrieval
We extract global descriptors with NetVLAD and find for each image the $k$ most similar ones. A larger $k$ improves the robustness of the localization for difficult queries but makes the matching more expensive. Using $k{=}10{-}20$ is generally a good tradeoff but $k{=}50$ gives the best results for the Aachen Day-Night dataset.

In [6]:
global_descriptors = extract_features.main(retrieval_conf, images, outputs)

[2023/04/16 15:18:58 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2023/04/16 15:18:58 hloc INFO] Found 328 images in root datasets/Droid_SLAM/left_images.
[2023/04/16 15:18:58 hloc.extractors.netvlad INFO] Downloading the NetVLAD model with `['wget', 'https://cvg-data.inf.ethz.ch/hloc/netvlad/Pitts30K_struct.mat', '-O', '/home/adam/.cache/torch/hub/netvlad/VGG16-NetVLAD-Pitts30K.mat']`.
--2023-04-16 15:18:58--  https://cvg-data.inf.ethz.ch/hloc/netvlad/Pitts30K_struct.mat
Resolving cvg-data.inf.ethz.ch (cvg-data.inf.ethz.ch)... 129.132.114.72
Connecting to cvg-data.inf.ethz.ch (cvg-data.inf.ethz.ch)|129.132.114.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554551295 (529M)
Saving to: ‘/home/adam/.cache/torch/hub/netvlad/VGG16-NetVLAD-Pitts30K.mat’

     0K .......... .......... .......... .......... ..........  0%  203K 44m25s
    50K ...

In [7]:
pairs_from_retrieval.main(global_descriptors, loc_pairs, num_matched=20)

[2023/04/16 15:40:08 hloc INFO] Extracting image pairs from a retrieval database.
[2023/04/16 15:40:09 hloc INFO] Found 6560 pairs.
